In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
import sklearn.neighbors
from sklearn import metrics
from geopy.distance import geodesic
from google.colab import drive
drive.mount('/content/drive',force_remount=True )
location='/content/drive/My Drive/CS 5228 Project/'

# Reading raw Training data and Testing data

In [ ]:
resale_data=pd.read_csv('{}train.csv/train.csv'.format(location))
df_r = pd.DataFrame(resale_data)

test_data=pd.read_csv('{}test.csv/test.csv'.format(location))
df_test = pd.DataFrame(test_data)

comm_center=pd.read_csv('{}auxiliary-data/sg-commerical-centres.csv'.format(location))
df_c = pd.DataFrame(comm_center)

primary_school=pd.read_csv('{}auxiliary-data/sg-primary-schools.csv'.format(location))
df_pschool=pd.DataFrame(primary_school)

demographics = pd.read_csv('{}auxiliary-data/sg-population-demographics.csv'.format(location))
df_demographics = pd.DataFrame(demographics)

hawkers = pd.read_csv('{}auxiliary-data/sg-gov-markets-hawker-centres.csv'.format(location))
df_hawkers = pd.DataFrame(hawkers)

malls = pd.read_csv('{}auxiliary-data/sg-shopping-malls.csv'.format(location))
df_malls = pd.DataFrame(malls)

mrt = pd.read_csv('{}auxiliary-data/sg-train-stations.csv'.format(location))
df_mrt = pd.DataFrame(mrt)

# Data Cleaning and Preprocessing Methods



1.   Remove elevation and eco-category as identified in EDA 
2.   Preprocess flat_type and remove hyphen(-)
3.   Create a new column storey_mid to convert the range into the mean 
4.   Split sale_year into sale_year(YYYY) and sale_month(MM)
5.   Calculate the lease period as this will determine how much time is left for the lease to end 
6.   Convert cateogrical flat_types and flat_model into ordinal attribute  
7.   Check if the town is a mature town or not
8.   Convert region into ordinal attribute
9.   Calculate demographics percentages 


In [ ]:
def data_preprocessing(X):
    X.drop(['elevation','eco_category'],axis='columns',inplace=True)
    X['flat_type']=X['flat_type'].str.replace(r'\-', ' ')
    X['storey_low'] = X['storey_range'].apply(lambda x: x.split(" ")[0]).astype('int')
    X['storey_high'] = X['storey_range'].apply(lambda x: x.split(" ")[2]).astype('int')
    X['storey_mid'] = ((X['storey_low']+X['storey_high'])/2).astype(int)
    X.drop(['storey_high','storey_low'],axis='columns',inplace=True)
    X['sale_year']=X['month'].apply(lambda x: x.split("-")[0]).astype(int)
    X['sale_month']=X['month'].apply(lambda x: x.split("-")[1]).astype(int)
    X['lease_period'] = (99- (X['sale_year'] - X['lease_commence_date']))

    flat_type_ord = { '1 room':1, '2 room': 2, '3 room': 3, '4 room': 4, '5 room': 5, 'executive': 6, 'multi generation':7}
    flat_model_ord={
    'type s2':3, 'type s1':3,'premium apartment loft':3,'dbss':3,'terrace':3,'premium maisonette':3,'multi generation':3,
    'maisonette':2,'model a maisonette':2,'improved maisonette':2,'apartment':2,'adjoined flat':2,'premium apartment':2,
    'improved':1,'model a2':1,'model a':1,'standard':1,'new generation':1,'simplified':1,'2 room':1}

    X['flat_type_ord']= X['flat_type'].map(flat_type_ord) #pd.Series(bedrooms[x] for x in df_train['flat_type'])
    X['flat_model_ord']=X['flat_model'].map(flat_model_ord) #pd.Series(bathrooms[x] for x in df_train['flat_type'])
    
    mature_towns=['ang mo kio','bedok','bishan','bukit merah','bukit timah','central area','clementi','geylang',
    'kallang/whampoa','marine parade','pasir ris','queenstown','serangoon','tampines','toa payoh',]

    non_mature_towns=['bukit batok','bukit panjang','choa chu kang','hougang','jurong east',
    'jurong west','punggol','sembawang','sengkang','woodlands','yishun']  #Removed Tengah as it is not in our dataset
    X['mature_town']=X['town'].isin(mature_towns).astype(int)
    
    dic = {'north region':1 , 'west region':2,'east region':3,'north-east region':4,'central region':5}
    X.region = x.region.map(dic)
    
    X = calculate_population_count(df_demographics, X)
    return X

def calculate_population_count(df_demographics, df_X):
    
    #group by planning-area and sum counts
    df_demo_grouped = df_demographics.groupby(['plannin_area', 'age_group'], as_index=False).sum()
    #take the upper value in age_group as age, eg: age = [15-19] = 15  
    df_demo_grouped["age"] = df_demo_grouped["age_group"].apply(lambda x: x.split("-")[0][0:2]).astype('int')

    #filter population based on age 
    #calculate counts demographics counts in each area 
    #calculate children(age <20), working adults(age:20-60), retired(age>60)
    children = df_demo_grouped.groupby('plannin_area').apply(lambda x: x[x["age"] < 20].sum()["count"])
    adults = df_demo_grouped.groupby('plannin_area').apply(lambda x: x[(x.age >= 20) & (x.age < 59)].sum()["count"])
    retired = df_demo_grouped.groupby('plannin_area').apply(lambda x: x[x["age"] >= 60].sum()["count"])

    #create a new dataframe with only planning_area and population counts
    df_population = pd.DataFrame()
    total_children = children.sum()
    total_adults = adults.sum()
    total_retired = retired.sum()

    df_population["childrenPerc"] = (children/total_children)*100
    df_population["adultsPerc"] = (adults/total_adults)*100
    df_population["retiredPerc"] = (retired/total_retired)*100
    df_population["planning_area"] = df_demo_grouped["plannin_area"].unique()

    df_hdb_population = df_X.merge(df_population, how='left', on='planning_area')
    return df_hdb_population

10. Calculate number of facilities in the given radius. The radius value for each of the auxiliary feature is identified in EDA 
11. Calculate the minimum distance between the HDB and closest auxiliary facility


In [ ]:
def calculate_geopy_distance(df_data, df_aux, radius,feature_title):
    list_of_lat = df_data['latitude']
    list_of_long = df_data['longitude']
    aux_lat = df_aux['lat']
    aux_long = df_aux['lng']
    # Zipping the respective Lat and Long lists together as a list of tuples
    list_of_coordinates = []
    for lat, long in zip(list_of_lat, list_of_long):
        list_of_coordinates.append((lat,long))
        
    list_of_aux_coordinates = []
    for lat, long in zip(aux_lat, aux_long):
        list_of_aux_coordinates.append((lat, long))

    list_of_dist_aux = []
    min_dist_aux = []
    n_neighbors = []
    for origin in list_of_coordinates:
        for destination in range(0, len(list_of_aux_coordinates)):
            list_of_dist_aux.append(geodesic(origin,list_of_aux_coordinates[destination]).meters)
        
        #Find the minimum distance from the list of distances calculated above
        shortest = (min(list_of_dist_mrt))
        #Find the number of neighboring facitilies within a given radius 
        neighboring_aux = list(filter(lambda x: x < radius,list_of_dist_aux))
        
        n_neighbors.append(len(neighboring_aux))
        min_dist_aux.append(shortest)
        list_of_dist_aux.clear()

    n_neighbors_title = "n_neighbors_{}".format(feature_title)
    df_data[n_neighbors_title] = n_neighbors

    min_dist_title = "min_dist_{}".format(feature_title)
    df_data[min_dist_title] = min_dist_aux

# Perform Preprocessing on Training Data

In [ ]:
train_data = data_preprocessing(df_r)
calculate_geopy_distance(train_data,df_mrt,1250,"mrt")
calculate_geopy_distance(train_data,df_malls,1300,"mrt")
calculate_geopy_distance(train_data,df_hawkers,1600,"mrt")
calculate_geopy_distance(train_data,df_pschool,1500,"mrt")
calculate_geopy_distance(train_data,df_c,2000,"mrt")
train_data.to_csv('{}/final codes/Data/clean_train.csv'.format(location))

# Perform Preprocessing on Test Data

In [ ]:
test_data = data_preprocessing(df_test)
calculate_geopy_distance(test_data,df_mrt,1250,"mrt")
calculate_geopy_distance(test_data,df_malls,1300,"mrt")
calculate_geopy_distance(test_data,df_hawkers,1600,"mrt")
calculate_geopy_distance(test_data,df_pschool,1500,"mrt")
calculate_geopy_distance(test_data,df_c,2000,"mrt")
test_data.to_csv('{}/final codes/Data/clean_test.csv'.format(location))